In [4]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pandas as pd
from collections import defaultdict
import os

In [ ]:
# os.cpu_count()

4

In [7]:
foundFile = False
while not foundFile:
    path = input('Enter your filepath to the data (.csv only, enter "exit" to end): ')
    if path == 'exit':
        break
    elif path[-4:] != '.csv':
        print("That's not a csv file!")
        continue
    try:
        df = pd.read_csv(path)
        foundFile = True
    except FileNotFoundError:
        print('File not found. Enter your filepath again.')
print('============ Column Names ============')
print(df.columns)
print('============ DataFrame ============')
print(df)

============ Column Names ============
Index(['student_id', 'age', 'gender', 'course', 'study_hours',
       'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality',
       'study_method', 'facility_rating', 'exam_difficulty', 'exam_score'],
      dtype='object')
============ DataFrame ============
       student_id  age  gender   course  study_hours  class_attendance  \
0               1   17    male  diploma         2.78              92.9   
1               2   23   other      bca         3.37              64.8   
2               3   22    male     b.sc         7.88              76.8   
3               4   20   other  diploma         0.67              48.4   
4               5   20  female  diploma         0.89              71.6   
...           ...  ...     ...      ...          ...               ...   
19995       19997   18   other      bba         6.50              71.3   
19996       19998   18    male    b.com         3.71              41.6   
19997       19999   

In [8]:
cols = df.columns[df.dtypes.eq('object')]
for c in cols:
    df[c] = df[c].astype('category').cat.codes


d = input('Enter the column(s) to drop (Enter "#NONE" if none): ')
li = d.split(' ')
print(li)
if li != ['#NONE']:
    for c in li:
        try:
            df.drop(columns=[c])
        except KeyError:
            print('Drop column failed, the column was', c)

scaler = StandardScaler()
scaled = scaler.fit_transform(df)

print(scaled)

['#NONE']
[[-1.73196262 -1.52044246 -0.00901182 ... -0.00966555 -0.21348541
  -0.19109485]
 [-1.73178942  1.10606684  1.2170862  ...  1.21382199  0.9290643
  -0.40793407]
 [-1.73161622  0.66831529 -0.00901182 ... -1.23315309  0.9290643
   1.46957626]
 ...
 [ 1.73178783 -0.64493936  1.2170862  ... -0.00966555 -1.35603512
   0.10507579]
 [ 1.73196103 -0.64493936 -0.00901182 ...  1.21382199  0.9290643
   0.87194621]
 [ 1.73213424 -0.20718781 -0.00901182 ...  1.21382199  0.9290643
   0.44884529]]


In [ ]:
while True:
    response = input('Enter the response variable: ')
    try:
        x = df.drop(columns=[response])
        y = df[response]
        
        break
    except KeyError:
        print('That column does not exist, please try again.')
        continue

while True:
    methods = input('Choose the regression methods you want to add (1. linear regression 2. polynomial 3. ridge 4. lasso 5. Logistic 6. Poisson 7. Quantile)')
    methods_list = methods.split(' ')
    have_methods = [False for _ in range(8)]
    have = False
    for m in methods_list:
        try:
            if int(m) >= 1 and int(m) <= 7:
                have_methods[int(m)-1] = True
                have = True
        except ValueError:
            print('Error!!!! The input ' + m + ' is not valid.')
    if have:
        break

cpu_cores = os.cpu_count()

linear_parems = defaultdict(list)


# fit_intercept: Whether to calculate the intercept for this model. If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).
# tol: The precision of the solution (coef_) is determined by tol which specifies a different convergence criterion for the lsqr solver.
# n_jobs: how much cpu cores will be used
# positive: force the coeffients to be positive or not

def do_linear():
    params_to_test = input('Enter the parameters you want to test. (1. fit_intercept 2. tol 3. n_jobs 4. positive')
    parems_li = params_to_test.split(' ')
    for p in parems_li:
        # fit_intercept
        if p == '1':
            fit_inter = input('Enter the values you want to try on '+'fit_intercept'+' (T/F)')
            fit_list = set(fit_inter.split(' '))
            if "T" in fit_list:
                linear_parems['fit_intercept'].append(True)
            elif "F" in fit_list:
                linear_parems['fit_intercept'].append(False)
        # tol
        elif p == '2':
            tol = set(input('Enter the values you want to try on '+'tol'+' (float numbers)').split(' '))
            for t in tol:
                try:
                    if float(t) < 0:
                        linear_parems['tol'].append(float(t))
                    else:
                        print(n +'is too big!!!!!!!!!!!!!!')
                except ValueError:
                    print(n +'was an invalid input!!!!!!!!!!!!!!')
        # n_jobs
        elif p == '3':
            nj = set(input('Enter the values you want to try on '+'n_jobs'+' (integer numbers)').split(' '))
            for n in nj:
                try:
                    if int(n) < 0:
                        if abs(int(n)) < cpu_cores:
                            linear_parems['n_jobs'].append(int(n))
                    elif int(n) >0:
                        if abs(int(n)) <= cpu_cores:
                            linear_parems['n_jobs'].append(int(n))
                except ValueError:
                    print(n +'was an invalid input!!!!!!!!!!!!!!')
        # positive
        elif p == '4':
            pos = set(input('Enter the values you want to try on '+'positive'+' (T/F)').split(' '))
            if "T" in pos:
                linear_parems['positive'].append(True)
            elif "F" in pos:
                linear_parems['positive'].append(False)

for i,v in enumerate(have_methods):
    if i == 0 and v:
        do_linear()
print(linear_parems)

defaultdict(<class 'list'>, {'1': [True], '2': [1000000000.0, 10000000000.0], '3': [-3, 1, 3, 2], '4': [True]})
